In [38]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import necessary libraries
import sys
from pathlib import Path
# Ensure the src directory is accessible
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get data

In [48]:
from get_data import get_data
data_split = get_data() 

Found local copy...
Loading...
Done!
100%|██████████| 13445/13445 [00:05<00:00, 2362.29it/s]

Data loaded and split successfully.
Train data samples: 9411
Validation data samples: 1344
Test data samples: 2690


# Featuring functions

In [54]:
from features import smiles_to_fp

# Prepare the data
def prepare_data(df):
    df['features'] = df['Drug'].apply(lambda x: smiles_to_fp(x))
    X = list(df['features'])
    y = df['Y'].values
    return X, y


# Prepare traing and test data

In [55]:
# make features
train_data = data_split['train']
X_train, y_train = prepare_data(train_data)

valid_data = data_split['valid']
X_valid, y_valid = prepare_data(valid_data)

test_data = data_split['test'] 
X_test, y_test = prepare_data(test_data)

# LTN first trial

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim

class LTNModel(nn.Module):
    def __init__(self):
        super(LTNModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1024, 64),  # Assuming input features size of 1024
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),  # binary classification
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)
    
    
    
# Instantiate the model
model = LTNModel()

# Prepare data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Setup optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define and run the training function
def train(model, X_train, y_train, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)  # Directly use the model
        loss = nn.BCELoss()(outputs, y_train)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Execute the training
train(model, X_train_tensor, y_train_tensor, optimizer)


# Set model to evaluation mode
nn_model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
predictions = (outputs > 0.5).float()  # Threshold probabilities to classify as 1 or 0

# Since you cannot use sklearn directly with tensors, you need to move data back to CPU and convert to numpy
predictions = predictions.cpu().numpy()
y_test_tensor = y_test_tensor.cpu().numpy()
eval(y_test_tensor, predictions)

ValueError: Using a target size (torch.Size([9411, 1, 1])) that is different to the input size (torch.Size([9411, 1])) is deprecated. Please ensure they have the same size.

# Logic Tensor Network (LTN)

In [ ]:
import ltn

In [ ]:
class LTNModel(torch.nn.Module):
    def __init__(self):
        super(LTNModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1024, 64),  # Assuming input features size of 1024
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),  # binary classification
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)
        
    
# Instantiate model and predicate
ltn_model = LTNModel()
#predicate_toxic = ltn.Predicate(ltn_model)


# LTN: Rules

In [ ]:
# Rule: Structural similarity implies similar toxicity
# Define a similarity measure (could be cosine similarity, Jaccard index, etc. for molecular fingerprints)
def similarity(x, y):
    return torch.cosine_similarity(x, y, dim=1)

# Logical rule
def rule_structural_similarity(data):
    for i in range(len(data)):
        for j in range(len(data)):
            # Enforce that similar compounds should have similar toxicity predictions
            ltn.axiom((similarity(data[i], data[j]) > 0.8) >> (ltn.equivalence(predicate_toxic(data[i]), predicate_toxic(data[j]))))


            
def structure_similarity(data, epsilon=0.85):
    # Assume a function that calculates pairwise similarity (e.g., cosine similarity)
    similarities = torch.mm(data, data.t())  # This is a simplistic way to compute similarities
    for i in range(len(data)):
        for j in range(len(data)):
            # Adding similarity rule: similar structures imply similar toxicity levels
            ltn.axiom((similarities[i, j] > epsilon) >> (ltn.equivalence(predicate_toxic(data[i]), predicate_toxic(data[j]))))

# Dummy function for presence of specific substructures (you need actual implementation based on your data)
def presence_of_toxic_substructures(data, indices_of_substructures):
    for i in range(len(data)):
        if any(data[i][idx] == 1 for idx in indices_of_substructures):
            ltn.axiom(predicate_toxic(data[i]))            


# Prepare Data

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)


# LTN: Training

In [ ]:
ltn_optimizer = optim.Adam(ltn_model.parameters(), lr=0.001)

# Define a training function
def ltn_train(predicate, X_train, y_train, optimizer, epochs=100):
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = predicate(X_train)  # Using the predicate directly
        loss = nn.BCELoss()(outputs.tensor, y_train.tensor)  # Accessing the tensor from the LTN Tensor
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")
            
# Step 7: Run the training
# Execute the training
ltn_train(predicate_toxic, X_train_ltn, y_train_ltn, ltn_optimizer)

In [ ]:

model.eval()
with torch.no_grad():
    test_outputs = predicate_toxic(X_test_term)
    test_predictions = (test_outputs > 0.5).float()
    accuracy = (test_predictions == y_test_term.tensor).float().mean()  # Compare against the tensor inside the term
    print(f"Test Accuracy: {accuracy.item()}")